# collections 相关用例

## defaultdict

In [1]:
# defaultdict

import collections

d = collections.defaultdict(int)
for i in range(10):
    d[i % 3] += 1
print(d)

defaultdict(<class 'int'>, {0: 4, 1: 3, 2: 3})


In [3]:
# 实际上，上述的代码类似于
class D(collections.defaultdict):
    def __missing__(self, key):
        return self.default_factory(key)
d = D(int)
for i in range(10):
    d[i % 3] += 1
print(d)

# 更简单的 lambda
def constant_factory(value):
    return lambda: value
d = collections.defaultdict(constant_factory(0))

for i in range(10):
    d[i % 3] += 1
print(d)

# 最后，需要注意的是，defaultdict 生效只在 `obj[key]` 这种调用的模式下，如果使用 `obj.get` 则不会触发。

defaultdict(<class 'int'>, {0: 4, 1: 4, 2: 5})
defaultdict(<function constant_factory.<locals>.<lambda> at 0x7fd83464cc80>, {0: 4, 1: 3, 2: 3})


## OrderedDict

In [8]:
od = collections.OrderedDict
d = od.fromkeys("abcde")

d.move_to_end("b", False) # 移动到队头
print(d)

d.move_to_end("d", True) # 默认情况下是 True, 即移动到队尾
print(d)

OrderedDict([('b', None), ('a', None), ('c', None), ('d', None), ('e', None)])
OrderedDict([('b', None), ('a', None), ('c', None), ('e', None), ('d', None)])


In [14]:
class LastUpdatedOrderedDict(collections.OrderedDict):
    ''' 每一个 key 新增或者修改操作都将其置于最后，可用于实现类似 LRU 的算法
    '''
    def __setitem__(self, key, value):
        if key in self:
            del self[key]
        super().__setitem__(key, value)

d = LastUpdatedOrderedDict.fromkeys("abcde")
print(d)
d['b'] = "B"
print(d)

# 在初始化时指定 key 的排序依据
d = {'banana': 3, 'apple': 4, 'pear': 1, 'orange': 2}
print(collections.OrderedDict(sorted(d.items(), key=lambda t: t[0]))) # sort by key
print(collections.OrderedDict(sorted(d.items(), key=lambda t: t[1]))) # sort by value

LastUpdatedOrderedDict([('a', None), ('b', None), ('c', None), ('d', None), ('e', None)])
LastUpdatedOrderedDict([('a', None), ('c', None), ('d', None), ('e', None), ('b', 'B')])
OrderedDict([('apple', 4), ('banana', 3), ('orange', 2), ('pear', 1)])
OrderedDict([('pear', 1), ('orange', 2), ('banana', 3), ('apple', 4)])


## Counter

In [22]:
# 初始化
c = collections.Counter()
c = collections.Counter({'red': 4, 'blue': 2})
c = collections.Counter(cats=4, dogs=8)
c = collections.Counter('hello')
print(c['l']) # 2

sum(c.values())                 # total of all counts
# c.clear()                       # reset all counts
list(c)                         # list unique elements
set(c)                          # convert to a set
dict(c)                         # convert to a regular dictionary
c.items()                       # convert to a list of (elem, cnt) pairs
collections.Counter(dict([("a", 1), ("b", 2)]))    # convert from a list of (elem, cnt) pairs

n = len(c)
print(c.most_common()[:-n-1:-1])       # n least common elements

b = collections.Counter('world')
print(b + c)                           # 对应的 count 相加（相减）
print(b | c)                           # 并集，相同取最大的 count
print(b & c)                           # 交集，相同取最小的 count
print(+c)                              # 相当于一个空的 counter 相加（或相减），但只保留正 count 的 key
print(-c)                              # 同上

2
[('o', 1), ('h', 1), ('e', 1), ('l', 2)]
Counter({'l': 3, 'o': 2, 'e': 1, 'd': 1, 'h': 1, 'r': 1, 'w': 1})
Counter({'l': 2, 'e': 1, 'd': 1, 'h': 1, 'r': 1, 'o': 1, 'w': 1})
Counter({'l': 1, 'o': 1})
Counter({'l': 2, 'e': 1, 'h': 1, 'o': 1})
Counter()


## deque

In [25]:
d = collections.deque(maxlen=3)
d.extend(range(3))
print(d)
d.append(4)
print(d)

try:
    d.insert(0, -1) # IndexError: deque already at its maximum size
except Exception as err:
    print(err)

d.pop()
print(d)
d.insert(0, -1) # deque([-1, 1, 2], maxlen=3)
print(d)

deque([0, 1, 2], maxlen=3)
deque([1, 2, 4], maxlen=3)
deque already at its maximum size
deque([1, 2], maxlen=3)
deque([-1, 1, 2], maxlen=3)


In [26]:
def tail(filename, n=10):
    ''' Return the last n lines of a file
    '''
    with open(filename) as f:
        return deque(f, n)

def delete_nth(d, n):
    ''' 删除第 n 个元素
    '''
    d.rotate(-n)
    d.popleft()
    d.rotate(n)

## nametuple

In [34]:
Point = collections.namedtuple('Point', ['x', 'y'])
# Point = collections.namedtuple('Point', 'x y') # 也是可以的

p = Point._make((1, 2))
print(p)
print(p.x == p[0])
print(isinstance(p, tuple))
print(isinstance(p, Point))

try:
    p.x = 2
except Exception as err:
    print(err)
    
p = p._replace(x=2)
print(p)

class Point(collections.namedtuple('Point', 'x y')):
    """ 这样则可以定义相应的方法
    
    下方 __slots__ = () 的设置则是为了不让对象生成 __dict__
    """
    __slots__ = ()

Point(x=1, y=2)
True
True
True
can't set attribute
Point(x=2, y=2)


## ChainMap

In [38]:
d1 = {1: 2, 3: 4, 5: 6}
d2 = {1: 1, 6: 7}

cm = collections.ChainMap(d1, d2)
print(cm.get(1))

cm = collections.ChainMap(d2, d1)
print(cm.get(1))

scm = cm.new_child()
print(scm)
pcm = cm.parents
print(pcm)

print(cm)
cm.maps[0], cm.maps[1] = cm.maps[1], cm.maps[0]
print(cm)

2
1
ChainMap({}, {1: 1, 6: 7}, {1: 2, 3: 4, 5: 6})
ChainMap({1: 2, 3: 4, 5: 6})
ChainMap({1: 1, 6: 7}, {1: 2, 3: 4, 5: 6})
ChainMap({1: 2, 3: 4, 5: 6}, {1: 1, 6: 7})


In [40]:
# python 变量查找顺序的模拟
import builtins
pylookup = collections.ChainMap(locals(), globals(), vars(builtins))


class DeepChainMap(collections.ChainMap):
    ''' 实现层级更新和删除
    '''
    def __setitem__(self, key, value):
        for mapping in self.maps:
            if key in mapping:
                mapping[key] = value
                return
        self.maps[0][key] = value

    def __delitem__(self, key):
        for mapping in self.maps:
            if key in mapping:
                del mapping[key]
                return
        raise KeyError(key)